##### Copyright 2019 The TensorFlow Authors.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Neural machine translation with attention

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import jieba

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

In [0]:
num_samples = 24000 
file_path = '/content/drive/My Drive/McMaster/projects/NLP/Data/English-Chinese/'
#list_file = 'list.txt'
cmn_file = 'cmn_s.txt'

In [0]:
def clean_en(text):
  text = unicode_to_ascii(text.lower().strip())
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "that is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"how's", "how is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"n't", " not", text)
  text = re.sub(r"n'", "ng", text)
  text = re.sub(r"'bout", "about", text)
  text = re.sub(r"'til", "until", text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
  text = '<start> ' + text + ' <end>'
  return text

def clean_zh(text):
  text = re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？：；.、~@#￥%……&*（）\n\t]+", "", text).strip()
  text = jieba.cut(text)
  text = '<start> ' + ' '.join(text) + ' <end>'
  return text

def process_sentence_zh(sentence):
  sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？：；.、~@#￥%……&*（）\n\t]+", "", sentence).strip()
  wordlist = jieba.lcut(sentence)
  return wordlist

In [0]:
def create_dataset():
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  word_pairs = []
  #with open(file_path+list_file, 'r') as lf:
  #  for data_path in lf:
  #    with open((file_path+data_path).strip(), 'r', encoding='utf-8') as f:
  with open((file_path+cmn_file).strip(), 'r', encoding='utf-8') as f:
      lines = f.read().split('\n')
  for line in lines[: min(num_samples, len(lines) - 1)]:
      #input_text, target_text = line.split('<s>')
      input_text, target_text, _ = line.split('\t')
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      target_text = '\t' + target_text + '\n'
      input_texts.append(clean_en(input_text))
      target_texts.append(clean_zh(target_text))    
      for char in input_text:
          if char not in input_characters:
              input_characters.add(char)
      for char in target_text:
          if char not in target_characters:
              target_characters.add(char)
      word_pairs.append([clean_en(input_text), clean_zh(target_text)])

  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))
  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])

  print('Number of samples:', len(input_texts))
  print('Number of unique input tokens:', num_encoder_tokens)
  print('Number of unique output tokens:', num_decoder_tokens)
  print('Max sequence length for inputs:', max_encoder_seq_length)
  print('Max sequence length for outputs:', max_decoder_seq_length)

  return zip(*word_pairs)

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset():
    # creating cleaned input, output pairs
    inp_lang, targ_lang = create_dataset()

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [60]:
# Try experimenting with the size of that dataset
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

Number of samples: 21204
Number of unique input tokens: 75
Number of unique output tokens: 2691
Max sequence length for inputs: 135
Max sequence length for outputs: 75


In [61]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

16963 16963 4241 4241


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [0]:
def convert2WordList(lang, tensor):
  word_list = []
  for t in tensor:
    if t!=0 and lang.index_word[t]!='<start>' and lang.index_word[t]!= '<end>':
      word_list.append(lang.index_word[t])
  return word_list

In [0]:
def convert2Sentence(lang, tensor):
  sentence = ''
  for t in tensor:
    if t!=0 and lang.index_word[t]!='<start>' and lang.index_word[t]!= '<end>':
      sentence +=' '+lang.index_word[t]
  return sentence

In [65]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
3 ----> i
27 ----> am
321 ----> able
5 ----> to
130 ----> speak
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
169 ----> 我能
30 ----> 说
2 ----> <end>


In [66]:
# convert validation list to list of word list
reference_translation_list = []
for i in target_tensor_val:
  reference_translation_list.append(convert2WordList(targ_lang, i))
print(reference_translation_list)

# convert input validation list to list of sentence:
test_sentence_list = []
for i in input_tensor_val:
  test_sentence_list.append(convert2Sentence(inp_lang,i))
print(test_sentence_list)

[['他游到', '不能', '游', '为止'], ['不', '我', '没', '去'], ['她', '给', '了', '我', '这个', '光碟'], ['她', '什么', '都', '不', '做', '只会', '抱怨'], ['我', '无法忍受', '它', '了'], ['她', '有', '几百', '本书'], ['这', '只', '狗', '是', '两个', '月', '前', '出生', '的'], ['她', '打', '了', '很', '多次', '电话', '给', '我'], ['这是', '什么', '啊'], ['这道', '题目', '没有', '一位', '老师', '会', '做'], ['他', '是', '我', '太太', '认识', '的', '一个', '人'], ['它', '是', '封', '长信'], ['对不起'], ['他', '问', '你', '什么'], ['这是', '一朵', '非常', '美丽', '的', '花'], ['这', '条', '裙子', '你', '穿着', '很', '合身'], ['你', '是', '在', '找', '工作', '吗'], ['我', '讨厌', '星期一'], ['你们', '两个', '在', '争论', '甚么'], ['除', '我', '以外', '每个', '人', '都', '在', '歌唱'], ['汤姆', '的', '降落伞', '没有', '打开'], ['战争', '还', '没', '结束'], ['他', '不是故意', '要', '伤害', '你'], ['我', '仅仅', '吃', '了', '口', '面包'], ['你', '喜欢', '果汁', '吗'], ['没有', '女人', '的话', '世界', '会', '是', '什么样'], ['那个', '是', '我', '的', '我', '不', '知道', '你', '的', '在', '哪里'], ['她', '在', '浴室'], ['没有', '必要', '着急'], ['他', '乘', '小船', '过河'], ['他', '不是', '来自', '北海道'], ['我会', '回来', '的'], ['我', '不能', '再

### Create a tf.data dataset

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [68]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 28]), TensorShape([64, 24]))

## Write the encoder and decoder model

Implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence. The below picture and formulas are an example of attention mechanism from [Luong's paper](https://arxiv.org/abs/1508.04025v5). 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

This tutorial uses [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) for the encoder. Let's decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU

The shapes of all the vectors at each step have been specified in the comments in the code:

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [70]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 28, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [72]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 28, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [74]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 10652)


## Define the optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
def evaluate_test_data(inputs):

    inputs = tf.convert_to_tensor([inputs])
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        if targ_lang.index_word[predicted_id] == '<end>':
            return result

        result += targ_lang.index_word[predicted_id] + ' '

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [0]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction

def get_bleu():
  # run the evaluation sentence list, and get the list of translation sentences
  translated_sentence_list=[]
  for i in input_tensor_val:
    sentence = evaluate_test_data(i)
    print(sentence)
    translated_sentence_list.append(sentence)
  
  #bleu, precisions, bp, ratio, translation_length, reference_length = compute_bleu(reference_translation_list, test_list,max_order=1) #translated_sentence_list)
  
  print('bleu -- {}'.format(corpus_bleu(reference_translation_list, translated_sentence_list)))

In [120]:
get_bleu()

他 游泳 他 不能 再 来 游泳 
没有 我 没 去 
她 给 了 我 一些 唱片 
她 没有 任何 音乐 
我 不能 忍受 它 还 坏 
她 的 想法 是 大事 
这 房间 里 发现 了 两个 衣橱 
她 替 我 一些 很 多次 
那 是 什么 
所有 的 老师 都 无法 解决 这个 问题 
他 和 我 的 认识 他 的 一位 我 的 一位 我 的 一位 我 的 一位 我 的 一位 我 的 一位 我 
这 是 我 很 远 
我错 我 
他 什么 也 没 做 什么 
这 是 个 美丽 的 树 
这 条 裙子 很 适合 你 的 
你 在 找 工作 吗 
我 讨厌 呼吸 
你 是 什么 呢 
每个 人 都 被 人 都 被 玛丽 都 被 玛丽 都 被 玛丽 都 被 玛丽 都 被 玛丽 都 被 玛丽 都 
汤姆 的 钱 还 没有 
战争 曾 是 没有 亮 了 
他 没有 跟 你 一直 高 
我 觉得 这 只 狗 租约 
你 喜欢 果汁 吗 
用 面粉 花 的 时候 怎么样 
你 为什么 说 你 的 地方 
她 在 洗澡 
没有 不 需要 着急 
他 一生 中 成为 了 一个 小 
他 不再 去过 北海道 
我会 打电话 
我 必须 停止 了 
我们 要 做 的 事情 要 做 的 人 
小心 脚下 
富士山 有 多久 
我 从没 见 过 他 把 钥匙 
她 搭 计程车 到 博物馆 
我 不 知道 他 把 最近 的 一个 都 是 谁 
他 在 那里 工作 
我 不 认为 我能 太 多 钱 
我 喜欢 弹钢琴 的 
他 没有 来 参加 这个 派对 
来 跟 我 一起 
汤姆 被 卡车 撞 于 监狱 
你 能 解决 这个 问题 呢 吗 
汤姆 正在 找 新 工作 
他 的 办公室 在 他家 附近 
他 不但 他 屈服 了 他 的 工作 的 事 
汤姆 终于 得到 了 正确 的 事 
不要 看不起 别人 说话 
我 睡着 了 时候 睡着 了 
我们 将 打破 重病 
她 坚持 坚持 取消 了 我 看见 那 一个 朋友 看见 
我 希望 他会 等 我 
你 建议 他们 来 唱歌 
你 为什么 都 不 说话 
拿走 吧 
没有 消息 是 值得 回答 
你 打算 干什么 
你 可以 寄希望于 可以 
我 请 给 mary 打开 窗户 
火警 发生 了 火灾 
这 是 我 身上 之间 的 原

In [116]:
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    #if batch % 100 == 0:
    print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    bleu_time = time.time()
    get_bleu()
    print('Time taken for bleu calculation {} sec\n'.format(time.time() - bleu_time))

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.0650
Epoch 1 Batch 1 Loss 0.0719
Epoch 1 Batch 2 Loss 0.0707
Epoch 1 Batch 3 Loss 0.0735
Epoch 1 Batch 4 Loss 0.0840
Epoch 1 Batch 5 Loss 0.0679
Epoch 1 Batch 6 Loss 0.0881
Epoch 1 Batch 7 Loss 0.0913
Epoch 1 Batch 8 Loss 0.0804
Epoch 1 Batch 9 Loss 0.0929
Epoch 1 Batch 10 Loss 0.0803
Epoch 1 Batch 11 Loss 0.0986
Epoch 1 Batch 12 Loss 0.0790
Epoch 1 Batch 13 Loss 0.0569
Epoch 1 Batch 14 Loss 0.0706
Epoch 1 Batch 15 Loss 0.0775
Epoch 1 Batch 16 Loss 0.0853
Epoch 1 Batch 17 Loss 0.0751
Epoch 1 Batch 18 Loss 0.0875
Epoch 1 Batch 19 Loss 0.0832
Epoch 1 Batch 20 Loss 0.0636
Epoch 1 Batch 21 Loss 0.0834
Epoch 1 Batch 22 Loss 0.0979
Epoch 1 Batch 23 Loss 0.0891
Epoch 1 Batch 24 Loss 0.0992
Epoch 1 Batch 25 Loss 0.0808
Epoch 1 Batch 26 Loss 0.0856
Epoch 1 Batch 27 Loss 0.0784
Epoch 1 Batch 28 Loss 0.0935
Epoch 1 Batch 29 Loss 0.0743
Epoch 1 Batch 30 Loss 0.0581
Epoch 1 Batch 31 Loss 0.0953
Epoch 1 Batch 32 Loss 0.0688
Epoch 1 Batch 33 Loss 0.0854
Epoch 1 Batch 34 Loss 0.

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [0]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = clean_en(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        result += targ_lang.index_word[predicted_id] + ' '

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [101]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [132]:
translate(u'I am a good boy')

Input: <start> i am a good boy <end>
Predicted translation: 我 是 个 好人 


In [0]:
translate(u'¿todavia estan en casa?')